# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Study data files
weather_metadata = "../output_data/weather_df.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Esperance,AU,1599700816,-33.87,121.90,99,72,64.40,20.80
1,Ushuaia,AR,1599700785,-54.80,-68.30,75,52,42.80,13.87
2,Arraial do Cabo,BR,1599700828,-22.97,-42.03,20,88,73.40,12.75
3,Ürümqi,CN,1599700829,43.80,87.60,98,58,59.00,4.47
4,Vaini,TO,1599700840,-21.20,-175.20,99,86,76.19,31.21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [10]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity (%)"].astype(float)

In [23]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp (F)"]<80,:]
second = first.loc[weatherdf["Max Temp (F)"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed (mph)"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness (%)"]==0,:]
fourth.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
59,Viranşehir,TR,1599700966,37.24,39.76,0,23,77.85,5.53
102,Lata,PT,1599701017,40.16,-8.33,0,74,71.01,2.33
197,Tiznit Province,MA,1599701131,29.58,-9.50,0,35,76.06,2.04
236,Hūn,LY,1599701178,29.13,15.95,0,45,76.08,5.14
258,Sirte,LY,1599701204,31.21,16.59,0,82,75.54,4.70


In [25]:
new_types_df = pd.DataFrame(weatherdf, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
59,Viranşehir,TR,37.24,39.76
102,Lata,PT,40.16,-8.33
197,Tiznit Province,MA,29.58,-9.50
236,Hūn,LY,29.13,15.95
258,Sirte,LY,31.21,16.59


In [27]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 59: Viranşehir.
Missing field/result... skipping.
------------
Retrieving Results for Index 102: Lata.
Missing field/result... skipping.
------------
Retrieving Results for Index 197: Tiznit Province.
Missing field/result... skipping.
------------
Retrieving Results for Index 236: Hūn.
Missing field/result... skipping.
------------
Retrieving Results for Index 258: Sirte.
Missing field/result... skipping.
------------
Retrieving Results for Index 313: Milton.
Missing field/result... skipping.
------------
Retrieving Results for Index 324: Poum.
Missing field/result... skipping.
------------
Retrieving Results for Index 338: Marrakesh.
Missing field/result... skipping.
------------
Retrieving Results for Index 345: Tabas.
Missing field/result... skipping.
------------
Retrieving Results for Index 350: Nouadhibou.
Missing field/result... skipping.
------------
Retrieving Results for Index 429: Ramnagar.
Missing field/result... skipping.
------------
Retrievin

In [32]:
hotel_df.head()


,City,Country,Lat,Lng
59,Viranşehir,TR,37.24,39.76
102,Lata,PT,40.16,-8.33
197,Tiznit Province,MA,29.58,-9.50
236,Hūn,LY,29.13,15.95
258,Sirte,LY,31.21,16.59


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [38]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig

NameError: name 'hotel_info' is not defined